In [24]:
# libraries
import sys; sys.path.insert(0, '..') # if you don't find it here, look one above
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.io as spio
import pickle
# stored one repo up in my fork of Spykes
from spykes.spykes.plot.neurovis import NeuroVis
from io_utils import *
from plotting_utils import * 

# settings
%load_ext autoreload
%autoreload 2
sns.set_context("talk")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# base paths/names
sess_names = np.load('sess_names.npy')
sess_name = sess_names[3]

base_path  = 'Y:\jbreda\ephys\post_sort_analysis\sorted_pre_bdata' 
beh_mat   = 'protocol_info.mat'
spks_mat  = 'ksphy_clusters_foranalysis.mat'


# create paths
sess_path = os.path.join(base_path, sess_name)
beh_path  = os.path.join(sess_path, beh_mat)
spks_path = os.path.join(sess_path, spks_mat)
fig_save_path = os.path.join(os.getcwd(), 'figures', 'neurovis', sess_name)

# load & wrangle
beh_df, spks_dict = load_and_wrangle(beh_path, spks_path, overwrite=True)


In [82]:
beh_df

,trial_num,hit_hist,delay,pair_hist,correct_side,prev_side,aud1_sigma,aud2_sigma,c_poke,end_state,hit_state,aud1_on,aud1_off,aud2_on,aud2_off,louder,first_sound
14,15,hit,2,4,RIGHT,LEFT,0.053144,0.019683,519.086082,524.267078,523.067086,519.136082,519.536082,521.536087,521.936083,aud_1,0.0531*
16,17,miss,6,6,LEFT,LEFT,0.002700,0.007290,533.121088,542.586081,541.255077,533.172076,533.572077,539.572077,539.972082,aud_2,0.0027
17,18,hit,6,3,RIGHT,LEFT,0.019683,0.007290,542.928090,552.206080,550.787081,542.979078,543.379079,549.379077,549.779079,aud_1,0.0197
18,19,hit,4,7,LEFT,RIGHT,0.007290,0.019683,556.319076,563.695078,562.421090,556.370079,556.769087,560.769079,561.169088,aud_2,0.0073
19,20,hit,2,1,RIGHT,LEFT,0.002700,0.001000,570.250080,575.399077,574.317085,570.300083,570.700080,572.701084,573.100081,aud_1,0.0027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,575,hit,6,4,RIGHT,RIGHT,0.053144,0.019683,6074.563072,6083.540076,6082.372079,6074.613073,6075.013075,6081.014077,6081.413073,aud_1,0.0531*
576,577,hit,4,2,RIGHT,RIGHT,0.007290,0.002700,6087.455079,6094.415079,6093.141071,6087.505079,6087.906078,6091.906082,6092.306080,aud_1,0.0073
577,578,hit,4,7,LEFT,RIGHT,0.007290,0.019683,6098.380082,6105.569074,6104.548081,6098.430082,6098.831073,6102.830094,6103.230084,aud_2,0.0073
579,580,hit,6,5,LEFT,LEFT,0.001000,0.002700,6112.223082,6121.463077,6120.364093,6112.273086,6112.674085,6118.673083,6119.073083,aud_2,0.0001*


### Separate spike times by trials

Create a list of list that is N trials long w/ spike info from cpoke to endstate for each trial. 

In [69]:
test_neuron = spks_dict['spk_times'][0]

In [111]:
# initialize
tr_times = np.zeros((len(beh_df),2))
tr_spk_L = []

# make counter bc idx is not continous
tr = 0

for idx, row in beh_df.iterrows():
    
    # grab start time 100 mas before c poke
    tr_start = row['c_poke'] - 0.001
    
    # create end time so delays have same length
    # NOTE: this is specific for analyzing delay period only
    if row['delay'] == 2:
        tr_end = tr_start + 5.5
    elif row['delay'] == 4:
        tr_end = tr_start + 7.7
    elif row['delay'] == 6:
        tr_end = tr_start + 9.9
    else:
        print('Trial delay is not 2, 4 or 6 seconds')
    
    # append start and end times
    tr_times[tr,:] = tr_start, tr_end
    tr+=1
    
    # grab spike time within trial window & append them
    tr_spk_L.append(test_neuron[np.logical_and(test_neuron >= tr_start, test_neuron <= tr_end)])
    

### Discretizing 

Take spk times x trials above, & given a bin size (1 ms) search in trial

In [50]:
# Initialize info for plotting
hit_df = beh_df[(beh_df['hit_hist'] == 'hit')]
aud1_df = hit_df[(hit_df['first_sound'] != 'psycho')]

In [53]:
bndl_dfs, df_names = deal_with_masking(spks_dict, aud1_df, sess_path, 'aud1_no_psycho', 30000)

Loading existing mask_dict...
Done loading.
ngood, first: 175
ngood, first: 175
ngood, second: 195
ngood, second: 195


In [58]:
# initilaize
events = ['aud1_on']
event_names = ['aud1_on']
window = [-500, 2300]
condition = 'first_sound'

In [66]:
beh_df

,trial_num,hit_hist,delay,pair_hist,correct_side,prev_side,aud1_sigma,aud2_sigma,c_poke,end_state,hit_state,aud1_on,aud1_off,aud2_on,aud2_off,louder,first_sound
14,15,hit,2,4,RIGHT,LEFT,0.053144,0.019683,519.086082,524.267078,523.067086,519.136082,519.536082,521.536087,521.936083,aud_1,0.0531*
16,17,miss,6,6,LEFT,LEFT,0.002700,0.007290,533.121088,542.586081,541.255077,533.172076,533.572077,539.572077,539.972082,aud_2,0.0027
17,18,hit,6,3,RIGHT,LEFT,0.019683,0.007290,542.928090,552.206080,550.787081,542.979078,543.379079,549.379077,549.779079,aud_1,0.0197
18,19,hit,4,7,LEFT,RIGHT,0.007290,0.019683,556.319076,563.695078,562.421090,556.370079,556.769087,560.769079,561.169088,aud_2,0.0073
19,20,hit,2,1,RIGHT,LEFT,0.002700,0.001000,570.250080,575.399077,574.317085,570.300083,570.700080,572.701084,573.100081,aud_1,0.0027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,575,hit,6,4,RIGHT,RIGHT,0.053144,0.019683,6074.563072,6083.540076,6082.372079,6074.613073,6075.013075,6081.014077,6081.413073,aud_1,0.0531*
576,577,hit,4,2,RIGHT,RIGHT,0.007290,0.002700,6087.455079,6094.415079,6093.141071,6087.505079,6087.906078,6091.906082,6092.306080,aud_1,0.0073
577,578,hit,4,7,LEFT,RIGHT,0.007290,0.019683,6098.380082,6105.569074,6104.548081,6098.430082,6098.831073,6102.830094,6103.230084,aud_2,0.0073
579,580,hit,6,5,LEFT,LEFT,0.001000,0.002700,6112.223082,6121.463077,6120.364093,6112.273086,6112.674085,6118.673083,6119.073083,aud_2,0.0001*


In [65]:
neuron.spiketimes

array([ 507.81889463,  507.83016075,  507.83856034, ..., 6178.54109253,
       6178.55675844, 6178.56985779])

In [63]:
window = np.zeros()
for ee in beh_df['aud1_on']:
    

519.136082
533.172076
542.979078
556.370079
570.300083
578.282084
597.041082
604.913075
617.170082
624.904081
644.27208
652.747077
663.39807
677.051088
690.197072
701.825079
711.269084
722.18808
732.025088
745.726089
764.549088
774.781072
785.477079
793.046089
823.783085
835.976079
846.355085
863.048078
877.27708
890.445079
902.449081
922.182079
929.619085
939.036081
953.435086
958.710072
971.35808
981.329084
986.343078
993.324085
1004.921077
1014.381075
1040.337089
1048.789078
1056.871079
1067.980082
1083.48107
1092.993088
1104.391077
1116.381084
1130.327087
1144.622079
1158.65307
1170.383074
1181.643086
1191.434078
1196.987077
1215.245089
1232.979079
1243.905085
1256.514079
1264.749088
1283.747082
1293.432078
1307.555077
1320.902077
1327.662079
1342.072074
1366.226089
1373.110083
1382.194083
1387.215087
1401.65708
1431.149089
1449.356083
1458.095079
1469.791078
1478.523083
1487.978082
1498.37608
1506.891083
1517.919073
1522.781078
1533.953082
1550.729083
1565.783075
1574.452089
1579.

In [ ]:
def event_center(spike_times, beh_df, event, window):
    
    
    

In [298]:
# initilaize
events = ['aud1_on']
windows = [[-1000,2300]]
condition = 'correct_side'

# get neurons into NeuroVis objects
neurons = initiate_neurons(spks_dict)
neuron = NeuroVis(spks_dict['spk_times'][0])

#single neuron debug
window = windows[0]
binsize = 100
df = bndl_dfs[df_names[0]]
phys_start = neuron.spiketimes[0]
phys_end = neuron.spiketimes[-1]
event = events[0]
conditions = 'correct_side'

In [13]:
neuron_rasters = get_neuron_rasters(neurons, events, windows, bndl_dfs, df_names)
neuron_psths = get_neuron_psths(neurons, events, windows, bndl_dfs, df_names, conditions=condition)

In [302]:
if not type(df) is dict:
        df = df.reset_index()
len(np.sum(raster['data'][0], axis = 1)), len(psth['data'][0]['mean'])

(14, 17)

In [176]:
if not type(df) is dict:
    df2 = df.reset_index()
wind = [np.floor(window[0]/binsize) * binsize, np.ceil(window[1]/binsize) * binsize]

In [177]:
trials = dict()
trials[0] = np.where(np.ones(np.size(df[event])))[0]
rasters = {
        'event': event,
        'conditions': conditions,
        'window': window,
        'binsize': binsize,
        'data': {},
    }

In [191]:
for cond_id in trials:
    # Select events relevant to this raster
    selected_events = df[event].iloc[trials[cond_id]]

    raster = []
    
    bin_template = 1e-3 * np.arange(window[0], window[1] + binsize, binsize)
    
    for event_time in selected_events:
        bins = event_time + bin_template



In [196]:
bin_template

array([-1. , -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1,  0. ,
        0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3])

In [195]:
for event_time in selected_events:
    bins = event_time + bin_template
    
    searchsorted_idx = np.sq

[5867.858354 5867.958354 5868.058354 5868.158354 5868.258354 5868.358354
 5868.458354 5868.558354 5868.658354 5868.758354 5868.858354 5868.958354
 5869.058354 5869.158354 5869.258354 5869.358354 5869.458354 5869.558354
 5869.658354 5869.758354 5869.858354 5869.958354 5870.058354 5870.158354
 5870.258354 5870.358354 5870.458354 5870.558354 5870.658354 5870.758354
 5870.858354 5870.958354 5871.058354 5871.158354]


In [212]:
event_time, event_time + 1e-3 * window[0], event_time + 1e-3 * window[1]

(5868.858354, 5867.858354, 5871.158354)